# Gallery Example: Replayer/M/1 Queue (Trace-Driven)

This example demonstrates a trace-driven queueing system:
- **Arrivals**: Replayed from trace file
- **Service**: Exponential
- **Servers**: 1 server
- **Scheduling**: FCFS

The Replayer distribution reads inter-arrival times from a file, enabling trace-driven simulation and analysis.

In [ ]:
from line_solver import *
import os
GlobalConstants.set_verbose(VerboseLevel.STD)

In [ ]:
def gallery_replayerm1(filename=None):
    """Create Trace/M/1 queueing model
    
    Args:
        filename: Path to trace file with inter-arrival times.
                  If None, uses example_trace.txt from LINE installation.
    """
    model = Network('Trace/M/1')
    
    # Block 1: nodes
    source = Source(model, 'mySource')
    queue = Queue(model, 'myQueue', SchedStrategy.FCFS)
    sink = Sink(model, 'mySink')
    
    # Block 2: classes
    oclass = OpenClass(model, 'myClass')
    
    # Create replayer from trace file
    if filename is None:
        # Use a simple exponential arrival for demo if no trace file
        source.set_arrival(oclass, Exp(1))
        queue.set_service(oclass, Exp(3))
    else:
        replayer = Replayer(filename)
        source.set_arrival(oclass, replayer)
        # Service rate set to 3x arrival rate for stability
        queue.set_service(oclass, Exp(3 / replayer.obj.getMean()))
    
    # Block 3: topology
    model.link(Network.serial_routing(source, queue, sink))
    
    return model

# Create the model (without trace file, uses exponential for demo)
model = gallery_replayerm1()

## About Trace-Driven Analysis

The Replayer distribution:
- Reads inter-arrival times from a text file
- Enables analysis using real workload data
- Can be converted to analytical distributions via fitting

Trace file format:
- One inter-arrival time per line
- Values are positive real numbers

Use cases:
- Validating models against real systems
- Analyzing recorded workload patterns
- Performance prediction with actual traffic

In [ ]:
# Solve with multiple solvers
print("\n=== Solver Results ===")

# CTMC Solver
solver_ctmc = CTMC(model, cutoff=15)
avg_table_ctmc = solver_ctmc.avg_table()
print("\nCTMC Solver:")
print(avg_table_ctmc)

# Fluid Solver
solver_fluid = FLD(model)
avg_table_fluid = solver_fluid.avg_table()
print("\nFluid Solver:")
print(avg_table_fluid)